In [645]:
import re
import emoji
import random
import pandas as pd
from pandas import read_csv
from opencc import OpenCC
import os, itertools
from itertools import groupby

In [646]:
# load data
map_ = read_csv('Map_Ramen_data.csv', encoding='utf8',index_col=0)
fb_ = read_csv('store_id_done.csv', encoding='utf8',index_col=0)
map_.head()

,store,address,discription,openTime,latitude,longtitute,mapReview,region,soup,transport
store_id,,,,,,,,,,
1,雞吉君拉麵とりよしくんラーメン雞白湯專門店,114台北市內湖區環山路一段33號,雞吉君是雞湯愛好者必朝聖的一間店，口味上從清淡到濃厚分別是淡麗->濃厚->特濃，三種口味各有...,週一 店休 PS：⚠️適逢每月雙數週休一、二週二~四 17:30~21:00(或售完)週五~...,25.086283,121.566969,Zhanyao Zhang-濃厚煮干雞拉麵 https://bit.ly/2QmFNoJ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #油拌麵 #淡麗 #雞白 #雞濃 #雞清 #煮干...,西湖國小／捷運西湖站
2,勝王,104台北市中山區林森北路306號,2020年8月22日重新開幕，本店為台灣第一個二毛作(原意為同一塊耕地一年內種植不同作物，在...,11:30~14:00、17:00~20:30,25.056086,121.525334,鄭傑瑞 - 橘醋清雞湯拉麵 https://bit.ly/38lDEnC 侯懿純 -...,北,#ramentw #台灣拉麵愛好會 #拉麵 #雞白湯 #薩索雞 #變化 #2019金賞,金光鐘錶
3,五之神製作所 台灣,110台北市信義區忠孝東路四段553巷6弄6號,為拉麵界較為少見的，以蝦作為基底的拉麵店，一共分為豚骨蝦湯麵、蝦沾麵與番茄蝦沾麵，其叉燒飯則...,週一~五 11:30~15:00、17:00~21:00週二 公休週六~日 11:30~21:00,25.042253,121.563041,Michael Lin - 雞白湯拉麵 https://bit.ly/360ZBqo ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #豚骨 #豚骨魚介 #蝦豚骨 #限定 #...,松菸／捷運市政府站 - 1號出口
4,Okaeriお帰り吃碗拉麵吧,106台北市大安區延吉街60號,年休中，10月26日(一)11:30再開。開拓台灣純正日本口味拉麵疆土的老店之一、與雞吉君為...,週二~週日: 11:30～15:00或（售完為止）17:30～21:00或（售完為止）每週一...,25.043711,121.553755,Queena Peng - 濃厚豚骨拉麵／辛口豚骨拉麵 https://bit.ly/3...,北,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #豚骨魚介 #辣味 #煎餃 #2019金賞,華視／捷運國父紀念館站
5,隱家拉麵-士林店,111台北市士林區中山北路五段505巷1號,開業於2019年10月，本店的風格為多樣化的湯頭，目前提供雞湯與豚骨湯二種湯頭，以及濃厚蝦沾...,11:30~14:30、16:30~22:00,25.093249,121.527470,風月 - 辛豚骨拉麵 https://bit.ly/37Ljcfw 高晉 - 黃金雞...,北,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #豚骨 #雞清湯 #豚骨蝦 #限定 #加...,捷運士林站


In [647]:
map_ = map_.sort_values(by=['store'])
map_ = map_.reset_index(drop=True)
map_ = map_.sort_index(axis=0 ,ascending=True)
map_ = map_.iloc[::-1]
map_ = map_.reset_index(drop=True)
map_.head()

,store,address,discription,openTime,latitude,longtitute,mapReview,region,soup,transport
0,龍魂屋台ラーメン,無提供地址資訊,雖是實體店面，仍以屋台風格設計店內有許多日系元素裝潢，猶如置身世外,18:00~02:00,NaN,NaN,吳俊青 - 塩味豚骨ラーメン http://bit.ly/2wbxeTk Lily ...,中,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,僑泰高中
1,黑潮拉麵·炸串專賣店,970花蓮縣花蓮市復興街71號,開幕於2017年11月，2020年5月1日遷移至現址，是花蓮在地首屈一指的拉麵專門店，深受在...,11:30~13:30、17:30~20:10,23.981691,121.606582,Youmu K YK - 黑蒜拉麵 https://bit.ly/39o2aVm 林...,東,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #炸串,花蓮市戶政事務所
2,黑曜麵屋,112台北市北投區尊賢街206巷2號,開業於2019年11月，以豚骨湯為主體的拉麵店，味道屬於台式風格，叉燒肉為此店的特色，詳情可...,11:00~21:00,25.115297,121.508182,由你來寫第一篇,北,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,曹家刈包
3,麵魂家,265宜蘭縣羅東鎮南門路158號,開幕於2015年6月，深耕宜蘭，屬於全台知名的在地拉麵店家，部份麵友們暱稱本店為「宜蘭王」「...,週二~五：12:00–14:30、17:00–20:20週六~日：12:00–20:20,24.673032,121.764848,蔡承澔 - 鹽味魚介薩索雞白湯 https://bit.ly/3fOPwyB Sta...,東,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #2019名店,羅東文化廣場對面
4,麵試十一次,105台北市松山區健康路68號之3號,開幕於2020年1月16日，主軸為雞湯味，詳情可察看下方網友食記。,11:30~14:30 、17:30~20:30週六、日 不定休,25.053285,121.553543,徐煜傑 - 鹽味黃金雞湯麵 https://bit.ly/2NwzPA7 Jimmy...,北,#ramentw #台灣拉麵愛好會 #拉麵 #雞湯,胖老爹健康路店


In [648]:
map_store = list(map_['store'])
map_store_lst = []

for store in map_store:
    store = store.lower()
    store = re.sub(r'[^\w\s]','',store)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬') \
                    .replace('鷹流東京豚骨拉麵極匠','鷹流極匠').replace('鷹流東京醬油拉麵蘭丸','鷹流蘭丸') \
                    .replace('#','').replace(' ','').replace(u'\u3000',u'').replace('柑橘shin','柑橘shinn') \
                    .replace('《','').replace('》','').replace('台','臺').replace('らーめん','拉麵').replace('の','之') \
                    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('ラーメン','拉麵') \
                    .replace('(','').replace(')','').replace('·','').replace('/','').replace('aqua2','').replace('麵鋪','麵舖') \
                    .replace('※','').replace('ぁ','あ').replace('²','2').replace('台湾','臺灣').replace('番茄','蕃茄') \
                    
    map_store_lst.append(store) 
                                                                              
#map_store_lst
map_['stem_map_store'] = map_store_lst
fb_store = list(fb_['stem_store'])
fb_store = sorted(list(set(fb_store)))
fb_store
all_store = map_store_lst + fb_store
shop_sorted = sorted(list(set(all_store)))
#shop_sorted

# Mapping store names (facebook and Google map)

In [649]:
### FIRST GROUPING
util_func = lambda x: x[0]  
first_sort = [list(ele) for i, ele in groupby(shop_sorted, util_func)] 
lst = list(itertools.chain(*first_sort))
#first_sort

In [650]:
### SECOND GROUPING
unique = []
second_sort = []

util_func = lambda x: x[1]
for item in first_sort:
    if len(item) > 1:
        temp = sorted(item, key = util_func)
        second_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        #second_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[1])])
    else:
        unique.append(item[0])

second_sort_len = []

for item_1 in second_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            second_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [651]:
#second_sort_len

In [652]:
### THIRD GROUPING
third_sort = []

util_func = lambda x: x[2]
for item in second_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵':
        third_sort.append([item])
    else:
        if len(item) > 1 and len(item[0]) >= 3:
            temp = sorted(item, key = util_func)
            third_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
            #third_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[2])])
        elif len(item) > 1 and len(item[0]) == 2:
            third_sort.append([item])
        else:
            unique.append(item[0])
        
third_sort_len = []

for item_1 in third_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            third_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [653]:
#third_sort_len

In [654]:
# Testing
lst = list(itertools.chain(*third_sort_len))
#print(len(lst)) 
#len(unique) 

In [655]:
### FOURTH GROUPING
fourth_sort = []

util_func = lambda x: x[3]
for item in third_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        fourth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) >= 4:
            temp = sorted(item, key = util_func)
            fourth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 3:
            fourth_sort.append([item])
        else:
            unique.append(item[0])
        
fourth_sort_len = []

for item_1 in fourth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fourth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [656]:
### FIFTH GROUPING
fifth_sort = []

util_func = lambda x: x[4]
for item in fourth_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵':
        fifth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) >= 5 and len(item[0]) > 7:
            temp = sorted(item, key = util_func)
            fifth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 4:
            fifth_sort.append([item])            
        elif len(item) > 1 and len(item[0]) >= 5 and len(temp[0]) <= 7:
            fifth_sort.append([item])
        else:
            unique.append(item[0])
            

fifth_sort_len = []

for item_1 in fifth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fifth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

In [657]:
#fifth_sort_len

In [658]:
### CHECK UNIQUE STORE NAME
#unique

In [659]:
done = []
unclassified_group = []

for item in fifth_sort_len:
    grouped = []
    unclassified = []
    grouped.append(item[0])
    for i in range(len(item)-1):
        if (item[0] in item[i+1]) or (item[0][:6] in item[i+1]):
            grouped.append(item[i+1])
        else:
            unclassified.append(item[i+1])
    done.append(grouped)
    if unclassified != []:
        unclassified_group.append(unclassified)

In [660]:
# Testing
lst = list(itertools.chain(*done))
#print(len(lst))
#print(len(unique))
clst = list(itertools.chain(*unclassified_group))
#print(len(clst))
#unclassified_group

In [661]:
for item in unclassified_group:
    if len(item) != 1:
        temp_grouped = []
        temp_unclassified = []
        temp_grouped.append(item[0])
        temp_unclassified.append(item[0])
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:len(item[0])//2] in item[i+1]):
                temp_grouped.append(item[i+1])
            else:
                temp_unclassified.append(item[i+1])
        done.append(temp_grouped)
        if len(temp_unclassified) >= 2:
            #print(temp_unclassified)
            unique.append(temp_unclassified[:])
    else:
        unique.append(item[0])

In [662]:
# Testing
lst = list(itertools.chain(*done))
#print(len(lst))
#print(len(unique))
#unique

In [663]:
### DICTIONARY OF STORE NAME
dict_done = {}
for i in range(len(done)):
    dict_done[i] = done[i]
#dict_done

In [664]:
### ANNOTATION_FACEBOOK
fb_store_id = []

for store in list(fb_['stem_store']):
    for key, v in dict_done.items():
        if store in v:
            fb_store_id.append(key)
            break
        elif store in unique:
            fb_store_id.append(unique.index(store)+500)
            break
        elif (store not in unique) and (key == list(dict_done.keys())[-1]):
            fb_store_id.append('9999')

In [665]:
### ANNOTATION_MAP
map_store_id = []

for store in list(map_['stem_map_store']):
    for key, v in dict_done.items():
        if store in v:
            map_store_id.append(key)
            break
        elif store in unique:
            map_store_id.append(unique.index(store)+500)
            break
        elif (store not in unique) and (key == list(dict_done.keys())[-1]):
            map_store_id.append('9999')

In [666]:
### OUTPUT CSV
fb_['fb_store_id'] = fb_store_id
map_['map_store_id'] = map_store_id
fb_ = fb_.drop(columns=['store_id'])
fb_ = fb_[['post_id', 'stores', 'stem_store','fb_store_id', 'create_on', 'ramen_name','fb_review']]
map_ = map_[['map_store_id','stem_map_store','store', 'address', 'discription', 'openTime', 'latitude', 'longtitute',
       'mapReview', 'region', 'soup', 'transport']]
#fb_.to_csv('fb_store_id_done.csv', index=True)
#map_.to_csv('map_store_id_done.csv', index=True)

In [667]:
#map_.columns